In [1]:
import cv2
import os
import numpy as np

input_file = 'VDB\L.mp4'   

# Open Video

In [2]:
cap = cv2.VideoCapture(input_file)
if not cap.isOpened():
    print("Error: Could not open video.")

# Get Video Properties

In [3]:
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
original_fps = cap.get(cv2.CAP_PROP_FPS)
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
print("Width: ",width)
print("Height: ",height)
print("Original FPS: ",original_fps)
print("Frame Count: ",frame_count)

Width:  640
Height:  360
Original FPS:  23.976023976023978
Frame Count:  2326


In [4]:
def get_frames(cap):
    frames = []
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    for i in range(frame_count):
        # Read a frame from the video
        ret, frame = cap.read()
        if not ret:
            print("Error: Could not read frame.")
            return frames

        # Save the frame to the list
        frames.append(frame)
        
    if not frames:
        print("No frames were saved.")
    
    return frames

In [5]:
cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
Frames = get_frames(cap)
cv2.imshow("Frame 0",Frames[0])
cv2.waitKey(0);
cv2.destroyAllWindows()

# Delete PNG Files in Directory

In [6]:
def delete_png_files(directory_path):
    # List all files in the specified directory
    for filename in os.listdir(directory_path):
        # Construct full file path
        file_path = os.path.join(directory_path, filename)
        try:
            # Check if it's a PNG file and remove it
            if os.path.isfile(file_path) and filename.lower().endswith('.png'):
                os.remove(file_path)
        except Exception as e:
            print(f'Failed to delete {file_path}. Reason: {e}')

# Read and Save Frames in Interval

In [7]:
def save_frames_as_vid(frames, frame_folder):
    # Create the folder to save frames if it doesn't exist
    if not os.path.exists(frame_folder):
        os.makedirs(frame_folder)

    frame_count = 0
    for i in range(len(frames)):
        # Save the frame as an image file
        frame_filename = os.path.join(frame_folder, f'frame_{frame_count:03d}.png')
        cv2.imwrite(frame_filename, frames[i])
        frame_count += 1

    #if not frames:
         #print("No frames were saved.")

In [8]:
delete_png_files('saved_frames')

In [9]:
save_frames_as_vid(Frames[50:100],"saved_frames")

# Create Video with Set of Frames

In [10]:
def save_vid(frames,output_file,fps):
    # Get frame dimensions
    height, width, _ = frames[0].shape

    # Define the codec and create VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_file, fourcc, fps, (width, height))

    # Write the frames to the new video file
    [out.write(frame) for frame in frames];

    # Release the video writer object
    out.release()

In [11]:
save_vid(Frames[100:200], "slice_vid.mp4", original_fps)

# Change FPS

In [12]:
save_vid(Frames[100:200],"slice_vid_new_fps.mp4",12)

# Play Frames

In [13]:
def play_frames(frames,fps):
    
    delay = int(1000/fps)
    print("Delay: ",delay)

    for frame in frames:
        # Display the frame
        cv2.imshow('Video Playback', frame)
        # Exit the playback if 'q' is pressed
        if cv2.waitKey(delay) & 0xFF == ord('q'):
            break
            
    cv2.destroyAllWindows()

In [14]:
play_frames(Frames[:100],24)

Delay:  41


# Optical Flow

In [15]:
def calculate_optical_flow(frames):
    #Convert to Grayscale
    prev_gray = cv2.cvtColor(frames[0], cv2.COLOR_BGR2GRAY)

    for i in range(len(frames)-1):
        #Convert the next frame to grayscale
        next_gray = cv2.cvtColor(frames[i+1], cv2.COLOR_BGR2GRAY)

        # Calculate the dense optical flow using Farneback method
        flow = cv2.calcOpticalFlowFarneback(prev_gray, next_gray, None, 0.5, 3, 15, 3, 5, 1.2, 0)

        # Visualize the optical flow
        hsv = np.zeros_like(frames[i])
        hsv[..., 1] = 255

        mag, ang = cv2.cartToPolar(flow[..., 0], flow[..., 1])
        hsv[..., 0] = ang * 180 / np.pi / 2
        hsv[..., 2] = cv2.normalize(mag, None, 0, 255, cv2.NORM_MINMAX)
        flow_rgb = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)

        # Display the original frame and the optical flow
        cv2.imshow('Original Frame', frames[i+1])
        cv2.imshow('Optical Flow', flow_rgb)

        # Wait for a key press to move to the next frame
        if cv2.waitKey(0) & 0xFF == ord('q'):
            break

        # Update the previous frame and grayscale image
        prev_gray = next_gray

    # Release the video capture object and close all OpenCV windows
    cv2.destroyAllWindows()

In [16]:
calculate_optical_flow(Frames[50:60])

In [44]:

def draw_optical_flow_vectors(flow, frame, step=16):
    h, w = frame.shape[:2]
    y, x = np.mgrid[step/2:h:step, step/2:w:step].reshape(2, -1).astype(int)
    fx, fy = flow[y, x].T

    # Create a mask to draw the vectors
    mask = np.zeros_like(frame)
    
    # Create line endpoints
    lines = np.vstack([x, y, x+fx, y+fy]).T.reshape(-1, 2, 2)
    lines = np.int32(lines + 0.5)

    # Draw lines and circles
    for (x1, y1), (x2, y2) in lines:
        cv2.line(mask, (x1, y1), (x2, y2), (0, 255, 0), 1)
        cv2.circle(frame, (x1, y1), 1, (0, 255, 0), -1)

    return cv2.add(frame, mask)

def calculate_optical_flow(video_file, n=5):
    # Open the video file
    cap = cv2.VideoCapture(video_file)
    if not cap.isOpened():
        print("Error: Could not open video.")
        return

    # Read the first frame and convert it to grayscale
    ret, prev_frame = cap.read()
    if not ret:
        print("Error: Could not read frame.")
        return
    prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)

    frame_count = 0

    while frame_count < n:
        # Read the next frame and convert it to grayscale
        ret, next_frame = cap.read()
        if not ret:
            break
        next_gray = cv2.cvtColor(next_frame, cv2.COLOR_BGR2GRAY)

        # Calculate the dense optical flow using Farneback method
        flow = cv2.calcOpticalFlowFarneback(prev_gray, next_gray, None, 
                                            0.5, 3, 15, 3, 5, 1.2, 0)

        # Draw the optical flow vectors on the frame
        flow_frame = draw_optical_flow_vectors(flow, next_frame)

        # Display the original frame with optical flow vectors
        cv2.imshow('Optical Flow Vectors', flow_frame)

        # Wait for a key press to move to the next frame
        if cv2.waitKey(0) & 0xFF == ord('q'):
            break

        # Update the previous frame and grayscale image
        prev_gray = next_gray
        frame_count += 1

    # Release the video capture object and close all OpenCV windows
    cap.release()
    cv2.destroyAllWindows()

video_file = input_file
#video_file = 'output_video.mp4'  # Path to the video file
calculate_optical_flow(video_file,N_F)


In [45]:
import os

def save_first_n_frames(video_file, output_file, frame_folder, n=5, fps=30):
    # Create the folder to save frames if it doesn't exist
    if not os.path.exists(frame_folder):
        os.makedirs(frame_folder)

    # Open the video file
    cap = cv2.VideoCapture(video_file)
    if not cap.isOpened():
        print("Error: Could not open video.")
        return

    frame_count = 0
    frames = []

    while frame_count < n:
        # Read a frame from the video
        ret, frame = cap.read()
        if not ret:
            print("Error: Could not read frame.")
            break

        # Save the frame to the list
        frames.append(frame)
        
        # Save the frame as an image file
        frame_filename = os.path.join(frame_folder, f'frame_{frame_count:03d}.png')
        cv2.imwrite(frame_filename, frame)
        
        frame_count += 1

    # Release the video capture object
    cap.release()

    if not frames:
        print("No frames were saved.")
        return

    # Get frame dimensions
    height, width, _ = frames[0].shape

    # Define the codec and create VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_file, fourcc, fps, (width, height))

    # Write the frames to the new video file
    for frame in frames:
        out.write(frame)

    # Release the video writer object
    out.release()
    print(f"Video saved to {output_file}")

video_file = input_file  # Path to the input video file
output_file = 'output_video.mp4'  # Path to the output video file
frame_folder = 'saved_frames'  # Folder to save individual frames
save_first_n_frames(video_file, output_file, frame_folder, N_F, fps=30)


Video saved to output_video.mp4


In [47]:
import cv2
import numpy as np

def draw_optical_flow_vectors(flow, frame, step=16):
    h, w = frame.shape[:2]
    y, x = np.mgrid[step/2:h:step, step/2:w:step].reshape(2, -1).astype(int)
    fx, fy = flow[y, x].T

    # Create a mask to draw the vectors
    mask = np.zeros_like(frame)
    
    # Create line endpoints
    lines = np.vstack([x, y, x+fx, y+fy]).T.reshape(-1, 2, 2)
    lines = np.int32(lines + 0.5)

    # Draw lines and circles
    for (x1, y1), (x2, y2) in lines:
        cv2.line(mask, (x1, y1), (x2, y2), (0, 255, 0), 1)
        cv2.circle(frame, (x1, y1), 1, (0, 255, 0), -1)

    return cv2.add(frame, mask)

def calculate_optical_flow_between_n_and_m(video_file, n, m):
    # Open the video file
    cap = cv2.VideoCapture(video_file)
    if not cap.isOpened():
        print("Error: Could not open video.")
        return

    frame_count = 0
    frames = []
    flows = []

    while True:
        # Read a frame from the video
        ret, frame = cap.read()
        if not ret:
            print("Error: Could not read frame.")
            break

        if frame_count >= n and frame_count < m:
            # Convert frame to grayscale
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            
            if len(frames) > 0:
                # Calculate optical flow using Farneback method
                flow = cv2.calcOpticalFlowFarneback(prev_gray, gray, None,
                                                    0.5, 3, 15, 3, 5, 1.2, 0)
                
                # Visualize optical flow on frame
                flow_frame = draw_optical_flow_vectors(flow, frame.copy())
                flows.append(flow_frame)

            frames.append(frame)
            
            prev_gray = gray

        frame_count += 1

        if frame_count >= m:
            break

    # Release the video capture object
    cap.release()

    if not frames:
        print("No frames were processed.")
        return

    # Display frames with optical flow
    for flow_frame in flows:
        cv2.imshow('Optical Flow Vectors', flow_frame)
        cv2.waitKey(0)

    cv2.destroyAllWindows()

video_file = input_file  # Path to the input video file
n = 50  # Start frame index
m = 100  # End frame index
calculate_optical_flow_between_n_and_m(video_file, n, m)


In [48]:
import cv2
import numpy as np

def find_pixels_with_most_changes(video_file, n, m, threshold=30):
    # Open the video file
    cap = cv2.VideoCapture(video_file)
    if not cap.isOpened():
        print("Error: Could not open video.")
        return

    frame_count = 0
    prev_frame = None

    while True:
        # Read a frame from the video
        ret, frame = cap.read()
        if not ret:
            print("Error: Could not read frame.")
            break

        if frame_count >= n and frame_count < m:
            if prev_frame is not None:
                # Convert frames to grayscale
                prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
                curr_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

                # Compute absolute difference between frames
                diff = cv2.absdiff(prev_gray, curr_gray)
                
                # Create a mask to highlight pixels with significant changes
                mask = np.zeros_like(diff)
                mask[diff > threshold] = 255

                # Show original frame and mask
                cv2.imshow('Original Frame', frame)
                cv2.imshow('Pixels with Most Changes', mask)
                cv2.waitKey(0)

            prev_frame = frame

        frame_count += 1

        if frame_count >= m:
            break

    # Release the video capture object
    cap.release()
    cv2.destroyAllWindows()

video_file = input_file # Path to the input video file
n = 50  # Start frame index
m = 100  # End frame index
threshold = 30  # Threshold for significant changes
find_pixels_with_most_changes(video_file, n, m, threshold)
